In [288]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df=pd.read_csv("eeg_baseline_level_9.csv")


In [289]:
df=df.dropna()

In [290]:
df

,Timestamp,TP9,AF7,AF8,TP10,SAS LEVEL
0,0.003906,-51.757812,-27.832031,-12.207031,-30.761719,9
1,0.007812,-47.363281,-26.367188,-11.718750,-31.738281,9
2,0.011719,-25.878906,-20.996094,-11.230469,-26.855469,9
3,0.015625,-20.019531,-25.878906,-13.671875,-19.042969,9
4,0.019531,-18.066406,-27.343750,-12.695312,-9.765625,9
...,...,...,...,...,...,...
30972,120.988281,-10.742188,-16.113281,-17.578125,-15.625000,9
30973,120.992188,-1.464844,-11.718750,-23.925781,-14.648438,9
30974,120.996094,-11.230469,-14.160156,-24.414062,-12.207031,9
30975,121.000000,-24.902344,-20.507812,-19.531250,-3.906250,9


In [291]:
import pandas as pd
import numpy as np
from scipy.signal import resample
def downsample_eeg(df, original_freq, new_freq):
    num_samples = len(df)
    new_num_samples = int(num_samples * new_freq / original_freq)
    
    resampled_df = pd.DataFrame()
    resampled_df['Timestamp'] = np.linspace(df['Timestamp'].min(), df['Timestamp'].max(), new_num_samples)
    
    for col in df.columns[1:]:
        resampled_df[col] = resample(df[col], new_num_samples)
    return resampled_df

# Downsample the data
original_freq = 256  # Original frequency
new_freq = 100    # New frequencyA
resampled_df = downsample_eeg(df, original_freq, new_freq)

# Print the resampled data
print(resampled_df)

        Timestamp        TP9        AF7        AF8       TP10  SAS LEVEL
0        0.003906 -45.820706 -25.233519 -15.877976 -27.572045        9.0
1        0.013907 -25.643885 -24.012690  -9.459993 -23.236114        9.0
2        0.023908 -15.453299 -28.180299 -22.585887  -5.554708        9.0
3        0.033909  -1.674276 -20.181770 -29.399433 -12.253373        9.0
4        0.043910  -8.068339 -13.361511  -9.095497  -3.732317        9.0
...           ...        ...        ...        ...        ...        ...
12095  120.963903 -28.142222 -23.943355 -13.616787 -20.031825        9.0
12096  120.973904 -17.025171 -22.642386 -26.745248 -17.212738        9.0
12097  120.983905 -16.891518 -13.779416 -14.578135  -9.242885        9.0
12098  120.993905 -13.653688 -13.680882 -20.117504 -20.731327        9.0
12099  121.003906 -12.475520 -15.797818 -22.203100  -6.339802        9.0

[12100 rows x 6 columns]


In [292]:
df2=resampled_df

In [1]:
from scipy.signal import butter, filtfilt, iirnotch
def butter_bandpass_filter(data, lowcut, highcut, fs, order=2):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

# Function to apply notch filter

# Example DataFrame
# Assuming df2 is your actual DataFrame with columns TP9, AF7, AF8, TP10

# Sample data generation (replace with actual data loading)

# Sampling rate
fs = 256.0  # Hz

# Bandpass filter parameters
lowcut = 4.0 # Hz
highcut = 8.0  # Hz

# Notch filter parameter

# Apply filters to each column and add SAS level
for col in ['TP9', 'AF7', 'AF8', 'TP10']:
    df2[col] = butter_bandpass_filter(df2[col], lowcut, highcut, fs)
# Print the updated DataFrame
print(df2)

NameError: name 'df2' is not defined

In [294]:
df2

,Timestamp,TP9,AF7,AF8,TP10,SAS LEVEL
0,0.003906,3.298293,6.580489,1.772804,5.183114,9.0
1,0.013907,22.162296,4.953513,3.485048,12.861367,9.0
2,0.023908,39.002946,4.574427,-4.196953,22.037519,9.0
3,0.033909,46.546486,11.376994,-4.659835,26.890489,9.0
4,0.043910,39.444780,17.834863,4.269277,22.250435,9.0
...,...,...,...,...,...,...
12095,120.963903,-10.105280,-11.072422,5.872449,-5.894717,9.0
12096,120.973904,-3.928237,-6.804982,4.410947,-7.808755,9.0
12097,120.983905,-0.167385,-0.856602,6.759564,-7.319786,9.0
12098,120.993905,-0.028836,-0.530502,8.521255,-7.933858,9.0


In [295]:
import pandas as pd
import numpy as np
from scipy.signal import welch
from scipy.stats import entropy
window_size = 10
segments = []
start_time = df2['Timestamp'].iloc[0]
end_time = df2['Timestamp'].iloc[-1]
for start in np.arange(start_time, end_time, window_size):
    end = start + window_size
    segment = df2[(df2['Timestamp'] >= start) & (df2['Timestamp'] < end)]
    if not segment.empty:
        segments.append(segment)
def calculate_psd(signal, fs=256):
    freqs, psd = welch(signal, fs=fs, nperseg=256)
    return psd

def calculate_de(signal):
    histogram, bin_edges = np.histogram(signal, bins=256, density=True)
    de = entropy(histogram)
    return de
features = []

for segment in segments:
    segment_features = {
        'start_time': segment['Timestamp'].iloc[0],
        'end_time': segment['Timestamp'].iloc[-1],
    }
    for col in ['TP9', 'AF7', 'AF8', 'TP10']:
        psd = calculate_psd(segment[col])
        de = calculate_de(segment[col])
        segment_features[f'{col}_psd'] = np.mean(psd)  # or any other summary statistic
        segment_features[f'{col}_de'] = de
    features.append(segment_features)
features_df = pd.DataFrame(features)
print(features_df)

    start_time    end_time    TP9_psd    TP9_de   AF7_psd    AF7_de   AF8_psd   
0     0.003906    9.994732   4.711889  4.115592  0.459076  4.940215  1.231709  \
1    10.004733   19.995558   2.540651  4.406915  0.268862  4.977715  0.369239   
2    20.005559   29.996385   5.223921  4.279685  0.336757  4.745062  0.467261   
3    30.006386   39.997211   5.740290  3.985278  0.343445  4.547282  0.650515   
4    40.007212   49.998038   3.162002  4.161105  0.305760  4.849019  0.512917   
5    50.008039   59.998865   6.248672  4.243683  0.305412  4.916112  0.372995   
6    60.008865   69.999691   6.861547  4.468996  0.732177  4.661422  0.899142   
7    70.009692   80.000518   4.889479  4.443875  0.745509  4.890194  1.183711   
8    80.010518   90.001344   8.254220  4.246317  0.828939  4.457080  1.500170   
9    90.011345  100.002171  10.580992  4.197492  0.630691  4.870914  1.310218   
10  100.012171  110.002997   3.745428  4.269122  0.334866  4.616105  1.001866   
11  110.012998  120.003824  

c:\Users\DIVYA\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 100, using nperseg = 100
  warnings.warn('nperseg = {0:d} is greater than input length '


In [296]:
features_df['SAS_Level']=9

In [297]:
features_df = features_df.iloc[:-1]

In [298]:
features_df

,start_time,end_time,TP9_psd,TP9_de,AF7_psd,AF7_de,AF8_psd,AF8_de,TP10_psd,TP10_de,SAS_Level
0,0.003906,9.994732,4.711889,4.115592,0.459076,4.940215,1.231709,4.334075,6.177087,4.014479,9
1,10.004733,19.995558,2.540651,4.406915,0.268862,4.977715,0.369239,4.697355,2.282502,4.135863,9
2,20.005559,29.996385,5.223921,4.279685,0.336757,4.745062,0.467261,4.487891,5.397160,3.983326,9
3,30.006386,39.997211,5.740290,3.985278,0.343445,4.547282,0.650515,4.153738,5.936112,3.801595,9
4,40.007212,49.998038,3.162002,4.161105,0.305760,4.849019,0.512917,4.671646,3.174151,4.179984,9
5,50.008039,59.998865,6.248672,4.243683,0.305412,4.916112,0.372995,4.761227,5.984022,4.179391,9
6,60.008865,69.999691,6.861547,4.468996,0.732177,4.661422,0.899142,4.483742,13.166461,4.271326,9
7,70.009692,80.000518,4.889479,4.443875,0.745509,4.890194,1.183711,4.647797,9.177493,4.388836,9
8,80.010518,90.001344,8.254220,4.246317,0.828939,4.457080,1.500170,4.446132,8.890825,4.161083,9
9,90.011345,100.002171,10.580992,4.197492,0.630691,4.870914,1.310218,4.693753,14.235376,4.158679,9


In [299]:
csv_file_path = 'eeg_data_with_sas_1_9.csv'
features_df.to_csv(csv_file_path, index=False)

print(f"DataFrame saved to {csv_file_path}")

DataFrame saved to eeg_data_with_sas_1_9.csv


In [300]:
df11=pd.read_csv("eeg_data_with_sas_1_1.csv")
df12=pd.read_csv("eeg_data_with_sas_1_2.csv")
df13=pd.read_csv("eeg_data_with_sas_1_3.csv")
df14=pd.read_csv("eeg_data_with_sas_1_4.csv")
df15=pd.read_csv("eeg_data_with_sas_1_5.csv")
df16=pd.read_csv("eeg_data_with_sas_1_6.csv")
df17=pd.read_csv("eeg_data_with_sas_1_7.csv")
df18=pd.read_csv("eeg_data_with_sas_1_8.csv")
df19=pd.read_csv("eeg_data_with_sas_1_9.csv")

In [301]:
combined_df = pd.concat([df11, df12, df13, df14, df15, df16, df17, df18, df19], ignore_index=True)

In [304]:
combined_df['Person']=9
combined_df['Type']='before'

In [305]:
combined_df

,start_time,end_time,TP9_psd,TP9_de,AF7_psd,AF7_de,AF8_psd,AF8_de,TP10_psd,TP10_de,SAS_Level,Person,Type
0,0.003906,9.994732,11.324338,4.237950,0.654774,5.026685,0.930921,4.565592,12.835278,4.106209,1,9,before
1,10.004733,19.995558,12.315015,4.142812,1.072952,4.666482,1.733340,4.390963,14.988503,4.203947,1,9,before
2,20.005559,29.996385,4.074828,3.868241,0.638001,4.934127,0.984895,4.334432,6.570659,3.778706,1,9,before
3,30.006386,39.997211,8.745940,4.084409,0.578450,4.991807,0.994004,4.572686,11.712933,4.156353,1,9,before
4,40.007212,49.998038,2.727995,4.163524,0.729425,4.924996,0.586539,4.598082,2.855702,4.048971,1,9,before
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,70.009692,80.000518,4.889479,4.443875,0.745509,4.890194,1.183711,4.647797,9.177493,4.388836,9,9,before
104,80.010518,90.001344,8.254220,4.246317,0.828939,4.457080,1.500170,4.446132,8.890825,4.161083,9,9,before
105,90.011345,100.002171,10.580992,4.197492,0.630691,4.870914,1.310218,4.693753,14.235376,4.158679,9,9,before
106,100.012171,110.002997,3.745428,4.269122,0.334866,4.616105,1.001866,4.571644,4.545979,4.175983,9,9,before


In [306]:
combined_csv_file_path = '9_combined_eeg_data_with_sas_2_before.csv'
combined_df.to_csv(combined_csv_file_path, index=False)